# Batch Predictions
https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker_batch_transform/tensorflow_cifar-10_with_inference_script/tensorflow-serving-cifar10-python-sdk.ipynb

and 

https://aws.amazon.com/blogs/machine-learning/performing-batch-inference-with-tensorflow-serving-in-amazon-sagemaker/

In [1]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Setup Input Data

In [2]:
# %store -r scikit_processing_job_name

In [3]:
# print(scikit_processing_job_name)

In [4]:
# # scikit_processing_job_s3_output_prefix = 'data'
# print('Previous Scikit Processing Job Name: {}'.format(scikit_processing_job_name))

In [5]:
prefix_predict = 'amazon-reviews-pds/tsv/'.format(bucket)

predict_s3_uri = 's3://{}/{}'.format(bucket, prefix_predict)

In [6]:
print(predict_s3_uri)

!aws s3 ls $predict_s3_uri/

s3://sagemaker-us-east-1-835319576252/amazon-reviews-pds/tsv/


# Setup Batch Transform Model

In [7]:
%store -r training_job_name

In [8]:
print(training_job_name)

tensorflow-training-2020-04-30-03-09-12-331


In [9]:
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz ./model.tar.gz

download: s3://sagemaker-us-east-1-835319576252/tensorflow-training-2020-04-30-03-09-12-331/output/model.tar.gz to ./model.tar.gz


In [10]:
!tar -xvzf ./model.tar.gz

tensorflow/
tensorflow/saved_model/
tensorflow/saved_model/0/
tensorflow/saved_model/0/saved_model.pb
tensorflow/saved_model/0/variables/
tensorflow/saved_model/0/variables/variables.index
tensorflow/saved_model/0/variables/variables.data-00000-of-00002
tensorflow/saved_model/0/variables/variables.data-00001-of-00002
tensorflow/saved_model/0/assets/
transformers/
transformers/fine-tuned/
transformers/fine-tuned/config.json
transformers/fine-tuned/tf_model.h5


In [11]:
!saved_model_cli show --all --dir ./tensorflow/saved_model/0/

2020-05-02 00:04:43.118873: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.0/lib64:/usr/local/cuda-10.0/extras/CUPTI/lib64:/usr/local/cuda-10.0/lib:/usr/local/cuda-10.0/efa/lib:/opt/amazon/efa/lib:/opt/amazon/efa/lib64:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/:
2020-05-02 00:04:43.118954: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.0/lib64:/usr/local/cuda-10.0/extras/CUPTI/lib64:/usr/local/cuda-10.0/lib:/usr/local/cuda-10.0/ef

In [24]:
from sagemaker.tensorflow.serving import Model

# If you change SAGEMAKER_TFS_DEFAULT_MODEL_NAME to something other than 'saved_model', you may see the dreaded ping error in the logs error
batch_env = {
  'SAGEMAKER_TFS_DEFAULT_MODEL_NAME': 'saved_model', # <== change this when using multi-model,
                                                     #     but watch out for the dreaded ping/ error 
                                                     #     if the model name doesn't exist
  'SAGEMAKER_TFS_ENABLE_BATCHING': 'true',
  'SAGEMAKER_TFS_BATCH_TIMEOUT_MICROS': '50000',
  'SAGEMAKER_TFS_MAX_BATCH_SIZE': '16'
}

batch_model = Model(
                    entry_point='batch_inference_csv.py',
                    source_dir='src_batch_inference_csv',
                    model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
                    role=role,
                    framework_version='2.1.0',
                    env=batch_env)

In [25]:
batch_predictor = batch_model.transformer(instance_count=1, 
                                          strategy='SingleRecord', 
                                          instance_type='local',                                      
#                                          instance_type='ml.m4.xlarge',
                                          accept='text/csv',
                                          assemble_with='Line',
                                          max_concurrent_transforms=64,
                                          max_payload=1, # This is in Megabytes (not number of records)
                                          env=batch_env)

# Start Batch Predictions

In [26]:
batch_predictor.transform(data=predict_s3_uri, 
                          split_type='Line',
                          compression_type='Gzip',
                          content_type='text/csv',
                          input_filter="$",
                          join_source='Input',
                          experiment_config=None)
#                          wait=True)

Local Mode only supports 1 ConcurrentTransform. Setting MaxConcurrentTransforms to 1


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 618, in run
    _stream_output(self.process)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 677, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 1

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 623, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpye2j7slk/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1



In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/sagemaker/home?region={}#/transform-jobs/{}?region={}&tab=Monitor">Batch Prediction Job</a></b>'.format(region, batch_predictor.latest_transform_job.job_name, region)))


In [ ]:
print('Waiting for batch prediction job: ' + batch_predictor.latest_transform_job.job_name)

batch_predictor.wait()

# Check Output Data

After the transform job has completed, download the output data from S3.

For each file "f" in the input data, we have a corresponding file "f.out" containing the predicted labels from each input row. 

We can compare the predicted labels to the true labels saved earlier.


In [ ]:
# Download the output data from S3 to local filesystem
batch_prediction_output_s3_uri = batch_predictor.output_path

# !mkdir -p ./batch_prediction_output

In [ ]:
%%bash 

aws s3 cp --recursive $batch_prediction_output_s3_uri/ ./batch_prediction_output

ls ./batch_prediction_output